In [1]:
import torch as t
import numpy as np
import preprocessing
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score
from sklearn.metrics import classification_report

c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TweetsDataset(Dataset):
    '''Generic dataset'''
    def __init__(self, csvPath, type):
        '''initialize the data and labels'''
        super().__init__()
        self.data, _, self.labels = preprocessing.preprocessDF(csvPath, type, getNERandPOS=False, applyStemming=True)
        self.vocabs = self.calculate_vocabs()
        self.vocab_size = len(self.vocabs)
        self.data_padding()
        self.replace_with_vocab_id()
        self.data = t.tensor(self.data)

    def __getitem__(self, index):
        ''' get the row item'''
        return self.data[index], self.labels[index]

    def data_padding(self):
        max_length = self.get_max_data_length()
        for data_row in self.data:
            if len(data_row) < max_length:
                padding = max_length - len(data_row)
                padding_list = [' '] * padding
                data_row += padding_list

    def get_max_data_length(self):
        max_length = -1e9
        for data_row in self.data:
            if len(data_row) > max_length:
                max_length = len(data_row)
        return max_length

    def calculate_vocabs(self):
        '''Calculate the vocabs book from the data'''
        vocabs_set = set()
        vocab_book = {}
        for tweet in self.data:
            vocabs_set.update(tweet)
        # here I started with token 1 as to leave zero token to the unknown word
        for i, word in enumerate(vocabs_set):
            vocab_book[word] = i + 1
        vocab_book['UNK'] = 0
        return vocab_book

    def replace_with_vocab_id(self):
        '''Replace the string word with its id in vocabs book'''
        for i, sentence in enumerate(self.data):
            self.data[i] = [self.vocabs[word] if word in self.vocabs else self.vocabs['UNK'] for word in sentence]


    def __len__(self):
        '''get the dataset size'''
        return len(self.data)
        

In [3]:
class CategoriesDataset(TweetsDataset):
    '''Tweets dataset with their categories'''
    def __init__(self, csvPath, type):
        '''initialize the data and labels'''
        super().__init__(csvPath, type)
        self.labels = t.tensor(self.labels)[:, 0]
        

In [4]:
class StancesDataset(TweetsDataset):
    '''Tweets dataset with their stances'''
    def __init__(self, csvPath, type):
        '''initialize the data and labels'''
        super().__init__(csvPath, type)
        self.labels = t.tensor(list(map(lambda x: 2 if x[1] == -1 else x[1], self.labels)))

In [5]:
class RNN(t.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_size = embedding_dim
        self.embedding = t.nn.Embedding(vocab_size, embedding_dim=embedding_dim)
        self.rnn = t.nn.RNN(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = t.nn.Linear(hidden_size, num_classes)

    def forward(self, input):
        input = self.embedding(t.tensor(input))
        out, _ = self.rnn(input)
        out = self.fc(out)
        return out


In [6]:
class TweetsClassifier():
    def __init__(self):
        pass
    def train(self):
        pass
    def predict(self):
        pass

In [34]:
class RnnTweetsClassifier(TweetsClassifier):
    def __init__(self, dataset, embedding_dim, hidden_size, num_layers, num_classes, epoch_size, learning_rate):
        super().__init__()
        self.dataset = dataset
        self.embedding_dim = embedding_dim
        self.epoch_size = epoch_size
        self.learning_rate = learning_rate
        self.model = RNN(dataset.vocab_size, embedding_dim, hidden_size, num_layers, num_classes)
        self.data_loader = DataLoader(dataset=dataset, batch_size=256, shuffle=True)
        self.criterion = t.nn.CrossEntropyLoss()
        self.optimizer = t.optim.Adam(params=self.model.parameters(), lr=self.learning_rate)
         
    def train(self):
        for epoch in range(self.epoch_size):
            total_model_acc = 0
            total_num_samples = 0
            for data in self.data_loader:
                train_data = data[:][0]
                labels = data[:][1]
                output = self.model(train_data)
                total_model_acc += (t.argmax(output, dim=2) == labels.repeat(output.shape[1], 1).transpose(0, 1)).sum()
                total_num_samples += len(labels) * output.shape[1]
                loss = self.criterion(t.reshape(output, (output.shape[0] * output.shape[1], -1)), labels.repeat(output.shape[1], 1).transpose(0, 1).flatten())
                self.optimizer.zero_grad()
                loss.backward()
                t.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1)
                self.optimizer.step()
            
            total_model_acc = total_model_acc / total_num_samples
            print(f"epoch num: {epoch} has accuracy: {100 * total_model_acc}")

    def predict(self, testLoader):
        with t.no_grad():
            predected = t.tensor([])
            real_labels = t.tensor([])
            for tweets in testLoader:
                test_data = tweets[:][0]
                labels = tweets[:][1]
                output = self.model(test_data)
                predected = t.cat((predected, t.argmax(output[:, -1, :], dim=1)))
                real_labels = t.cat((real_labels, labels))
            return predected, real_labels
            


In [8]:
class Evaluation():
    def __init__(self, predicted, real_labels):
        self.pred = predicted
        self.labels = real_labels
        accuracy = accuracy_score(real_labels, predicted)
        recall = recall_score(real_labels, predicted)
        precision = precision_score(real_labels, predicted)
        f1 = f1_score(real_labels, predicted)
        print(f'''  Accuracy: {accuracy}
                    Recall: {recall}
                    Precision: {precision}
                    F1-Score: {f1}''')

In [73]:
# load the dataset and dataloader
dataset = StancesDataset('./dataset/train.csv', 'train')
tweet_classifier = RnnTweetsClassifier(dataset, 50, 50, 3, 3, 10, 0.001)

In [74]:
# train the stances model
tweet_classifier.train()

C:\Users\omara\AppData\Local\Temp\ipykernel_6140\3290526726.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = self.embedding(t.tensor(input))


epoch num: 0 has accuracy: 76.29442596435547
epoch num: 1 has accuracy: 79.24826049804688
epoch num: 2 has accuracy: 79.2498550415039
epoch num: 3 has accuracy: 79.2498550415039
epoch num: 4 has accuracy: 79.2498550415039
epoch num: 5 has accuracy: 79.24638366699219
epoch num: 6 has accuracy: 79.23770904541016
epoch num: 7 has accuracy: 79.3157730102539
epoch num: 8 has accuracy: 79.21849060058594
epoch num: 9 has accuracy: 79.04344177246094


In [75]:
testset = StancesDataset('./dataset/dev.csv', 'test')
testloader = DataLoader(dataset=testset, batch_size=256, shuffle=True)

In [76]:
predicted, labels = tweet_classifier.predict(testloader)
print(classification_report(labels, predicted))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       126
         1.0       0.80      1.00      0.89       804
         2.0       0.00      0.00      0.00        70

    accuracy                           0.80      1000
   macro avg       0.27      0.33      0.30      1000
weighted avg       0.65      0.80      0.72      1000



C:\Users\omara\AppData\Local\Temp\ipykernel_6140\3290526726.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = self.embedding(t.tensor(input))
c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Us

In [84]:
# load the dataset and dataloader
categories_dataset = CategoriesDataset('./dataset/train.csv', 'train')
gategories_classifier = RnnTweetsClassifier(categories_dataset, 50, 50, 3, 10, 10, 0.001)

In [85]:
gategories_classifier.train()

C:\Users\omara\AppData\Local\Temp\ipykernel_6140\3290526726.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = self.embedding(t.tensor(input))


epoch num: 0 has accuracy: 46.31070327758789
epoch num: 1 has accuracy: 51.577598571777344
epoch num: 2 has accuracy: 51.66851806640625
epoch num: 3 has accuracy: 51.69786071777344
epoch num: 4 has accuracy: 51.71361541748047
epoch num: 5 has accuracy: 51.72734832763672
epoch num: 6 has accuracy: 51.73183059692383
epoch num: 7 has accuracy: 51.74093246459961
epoch num: 8 has accuracy: 51.82809829711914
epoch num: 9 has accuracy: 52.23774719238281


In [82]:
categories_testset = CategoriesDataset('./dataset/dev.csv', 'test')
categories_testloader = DataLoader(dataset=categories_testset, batch_size=256, shuffle=True)

In [83]:
predicted, labels = gategories_classifier.predict(categories_testloader)
print(classification_report(labels, predicted))

              precision    recall  f1-score   support

         0.0       0.52      0.47      0.50       545
         1.0       0.00      0.00      0.00       145
         2.0       0.00      0.00      0.00        82
         3.0       0.00      0.00      0.00        20
         4.0       0.00      0.00      0.00        15
         5.0       0.00      0.00      0.00        10
         6.0       0.00      0.00      0.00         2
         7.0       0.14      0.55      0.22       128
         8.0       0.00      0.00      0.00        36
         9.0       0.00      0.00      0.00        17

    accuracy                           0.33      1000
   macro avg       0.07      0.10      0.07      1000
weighted avg       0.30      0.33      0.30      1000



C:\Users\omara\AppData\Local\Temp\ipykernel_6140\3290526726.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = self.embedding(t.tensor(input))
c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Us